In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
train=pd.read_csv('train.csv')

In [21]:
X=train.drop('label',axis=1)

In [24]:
y=train['label']

In [25]:
y.shape

(80213,)

In [34]:
X_test=pd.read_csv('test.csv')

In [35]:
X_test

,id,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
0,276f619c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8ced6677,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,af549c74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,59170e1b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ac25b7f0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,7ee547cf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8900,dfc2f44d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8901,543715a7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8902,47d51e67,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

# Assuming X and y are your training data
# And you have a separate test set (X_test, y_test)

# Convert labels to categorical (one-hot encoding)
y_cat = tf.keras.utils.to_categorical(y - 1, num_classes=36)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Reshape data into sequences (assuming 50 time steps with 50 features each)
# However, this doesn't fit the 2500 feature structure, so let's adjust it
# For demonstration, let's assume we have sequences of 50 time steps with some features

# Assuming each sample has 2500 features, let's reshape into sequences of 50 time steps
# This means we need to adjust the number of features per time step
num_features_per_step = 50
X_train_reshaped = X_train.values.reshape(-1, 50, num_features_per_step)
X_val_reshaped = X_val.values.reshape(-1, 50, num_features_per_step)

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(50, num_features_per_step)),
    Dense(64, activation='relu'),
    Dense(36, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=500, validation_data=(X_val_reshaped, y_val))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val_reshaped, y_val)
print(f'Validation accuracy: {accuracy:.2f}')

# Predict classes for validation data
predictions = model.predict(X_val_reshaped)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Convert back to class numbers

# Evaluate accuracy manually
manual_accuracy = accuracy_score(np.argmax(y_val, axis=1) + 1, predicted_classes)
print(f'Manual accuracy check: {manual_accuracy:.2f}')


Epoch 1/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - accuracy: 0.0587 - loss: 3.4816 - val_accuracy: 0.1576 - val_loss: 2.8455
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.1930 - loss: 2.7056 - val_accuracy: 0.2623 - val_loss: 2.4248
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.2883 - loss: 2.3111 - val_accuracy: 0.3254 - val_loss: 2.1730
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.3492 - loss: 2.0545 - val_accuracy: 0.3699 - val_loss: 2.0047
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.3996 - loss: 1.8936 - val_accuracy: 0.4045 - val_loss: 1.8758
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.4390 - loss: 1.7445 - val_accuracy: 0.4341 - val_loss: 1.7886
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.4704 - loss: 1.6436 - val_accuracy: 0.4632 - val_loss: 1.7046
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.4968 - loss: 1.5605 - 

In [31]:
import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

# Assuming X and y are your training data
# And you have a separate test set (X_test, y_test)

# Convert labels to categorical (one-hot encoding)
y_cat = tf.keras.utils.to_categorical(y - 1, num_classes=36)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Reshape data into images (50x50)
X_train_reshaped = X_train.values.reshape(-1, 50, 50, 1)
X_val_reshaped = X_val.values.reshape(-1, 50, 50, 1)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(36, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=45, batch_size=500, validation_data=(X_val_reshaped, y_val))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val_reshaped, y_val)
print(f'Validation accuracy: {accuracy:.2f}')

# Predict classes for validation data
predictions = model.predict(X_val_reshaped)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Convert back to class numbers

# Evaluate accuracy manually
manual_accuracy = accuracy_score(np.argmax(y_val, axis=1) + 1, predicted_classes)
print(f'Manual accuracy check: {manual_accuracy:.2f}')


Epoch 1/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 42s 317ms/step - accuracy: 0.0370 - loss: 4.6499 - val_accuracy: 0.0727 - val_loss: 3.4282
Epoch 2/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 45s 351ms/step - accuracy: 0.1092 - loss: 3.2630 - val_accuracy: 0.3228 - val_loss: 2.3090
Epoch 3/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 28s 219ms/step - accuracy: 0.4481 - loss: 1.8373 - val_accuracy: 0.7022 - val_loss: 0.9939
Epoch 4/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 204ms/step - accuracy: 0.7398 - loss: 0.8568 - val_accuracy: 0.7998 - val_loss: 0.6786
Epoch 5/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 27s 208ms/step - accuracy: 0.8297 - loss: 0.5596 - val_accuracy: 0.8399 - val_loss: 0.5342
Epoch 6/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 204ms/step - accuracy: 0.8706 - loss: 0.4244 - val_accuracy: 0.8675 - val_loss: 0.4507
Epoch 7/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 204ms/step - accuracy: 0.9031 - loss: 0.3234 - val_accuracy: 0.8854 - val_loss: 0.3932
Epoch 8/45
129/129 ━━━━━━━━━━━━━━━━━━━━ 28s 216ms/step - accuracy: 0.9174 - loss: 0

In [41]:
import pandas as pd
import numpy as np

# Assuming X_test is a DataFrame with an 'id' column
# and the rest are features

# Split X_test into id and features
id_column = X_test[['id']]
features = X_test.drop('id', axis=1)

# Convert features to NumPy array and reshape into sequences
X_test_array = features.to_numpy()
X_test_reshaped = X_test_array.reshape(-1, 50, 50,1)

# Predict using the reshaped features
predictions = model.predict(X_test_reshaped)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Convert back to class numbers

# Combine id with predictions
output_df = pd.DataFrame({
    'id': id_column['id'],
    'label': predicted_classes
})

print(output_df)


279/279 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
            id  label
0     276f619c     21
1     8ced6677     11
2     af549c74     21
3     59170e1b     31
4     ac25b7f0     19
...        ...    ...
8899  7ee547cf      2
8900  dfc2f44d     29
8901  543715a7     34
8902  47d51e67      7
8903  0c042f16     15

[8904 rows x 2 columns]


In [42]:
output_df

,id,label
0,276f619c,21
1,8ced6677,11
2,af549c74,21
3,59170e1b,31
4,ac25b7f0,19
...,...,...
8899,7ee547cf,2
8900,dfc2f44d,29
8901,543715a7,34
8902,47d51e67,7


In [43]:
output_df.to_csv('submission.csv', index=False)